## Import modules and load data

In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
df_notes = pd.read_csv('../data/original/NOTEEVENTS.csv', low_memory=False)

In [3]:
df_icustays = pd.read_csv('../data/original/ICUSTAYS.csv')

In [4]:
multi_to_patid = pd.read_csv('../data/processed/multi_to_patid.csv')

## Filter & clean table

In [5]:
rel_cats = ['SUBJECT_ID','HADM_ID','CHARTDATE','CHARTTIME','CATEGORY','TEXT']
df_notes = df_notes[rel_cats]
# Why have notes if they have no text
df_notes.dropna(subset=['TEXT'], inplace=True)
# Make it friendly for datetime comparisons later on
df_notes.CHARTDATE = df_notes.CHARTDATE.astype('datetime64[ns]')
df_notes.CHARTTIME = df_notes.CHARTTIME.astype('datetime64[ns]')

Create multi-level index

In [6]:
df_notes.set_index(['SUBJECT_ID','HADM_ID'], inplace=True)

Clean up category names

In [7]:
df_notes.loc[df_notes.CATEGORY == 'Discharge summary', 'CATEGORY'] = 'Discharge'
df_notes.loc[df_notes.CATEGORY == 'Physician ', 'CATEGORY'] = 'Physician'
df_notes.loc[df_notes.CATEGORY == 'Respiratory ', 'CATEGORY'] = 'Respiratory'

Select common categories

In [8]:
relevant_cats = ["Discharge","Nursing/other","Radiology","Nursing","ECG","Physician","Echo","Respiratory","Nutrition","General"]
df_notes = df_notes.loc[df_notes.CATEGORY.map(lambda x: x in relevant_cats)]

## Clean up df_icustays

In [9]:
# Convert from 'object' to 'datetime'
df_icustays.INTIME = df_icustays.INTIME.astype('datetime64[ns]')
df_icustays.OUTTIME = df_icustays.OUTTIME.astype('datetime64[ns]')
# Create date columns in case row in question doesn't have a 'CHARTTIME' for comparison
df_icustays['INDATE'] = df_icustays.INTIME.dt.date.astype('datetime64[ns]')
df_icustays['OUTDATE'] = df_icustays.OUTTIME.dt.date.astype('datetime64[ns]')
# Select only the first ICU stay for a patient across all admissions
df_icustays = df_icustays.sort_values(by='INTIME').groupby(['SUBJECT_ID']).first()
# Select the minimal information needed to match ICU stays to clinical notes
df_icustays = df_icustays[["HADM_ID","ICUSTAY_ID","INTIME","INDATE","OUTTIME","OUTDATE"]]
# Impossible to verify notes belong to an ICU stay without a time
df_icustays.dropna(subset=['INTIME','OUTTIME'], inplace=True)
# Each index is a single patient's first ICU visit (we removed multiple ICU visits)
df_icustays.reset_index(inplace=True)
df_icustays.set_index(['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], inplace=True)

### Limit to first icustay

In [10]:
df_notes_icu = df_icustays.copy()
df_notes_icu = df_notes_icu.join(df_notes, how='inner')

In [11]:
# Date comparison
date_val = (df_notes_icu.CHARTDATE >= df_notes_icu.INDATE) & (df_notes_icu.CHARTDATE <= df_notes_icu.OUTDATE)
# Time comparison
time_val = (df_notes_icu.CHARTTIME >= df_notes_icu.INTIME) & (df_notes_icu.CHARTTIME <= df_notes_icu.OUTTIME)
# Valid?
datetime_val = (df_notes_icu.CHARTTIME.isna() & date_val) | (~df_notes_icu.CHARTTIME.isna() & time_val) 
# Filter rows
df_notes_icu = df_notes_icu.loc[datetime_val]

### Match Cohorts to pre-selected cohort

In [12]:
multi_to_patid.set_index(['subject_id','hadm_id','icustay_id'], inplace=True)
df_notes_icu.index.names = ['subject_id','hadm_id','icustay_id']

In [13]:
df_notes_icu = multi_to_patid.join(df_notes_icu, how='left')

In [14]:
df_notes_icu.dropna(subset=['INTIME','INDATE'], inplace=True)

In [15]:
df_notes_icu.reset_index(inplace=True)
df_notes_icu.set_index('pat_id', inplace=True)
df_notes_icu.drop(['subject_id','hadm_id','icustay_id'], axis=1, inplace=True)

### Split Static from Time series notes

In [16]:
df_notes_static = df_notes_icu.loc[df_notes_icu.CATEGORY == 'Discharge', :].copy()
df_notes_ts = df_notes_icu.loc[~(df_notes_icu.CATEGORY == 'Discharge'), :].copy()

### Trim static notes

In [17]:
df_notes_static["TEXT_LEN"] = df_notes_static.TEXT.map(lambda s: len(s))
df_notes_static = df_notes_static.sort_values(by='TEXT_LEN', ascending=False).groupby('pat_id').first()

In [18]:
df_notes_static = df_notes_static[['TEXT']]

### Handle ECG and Echo

In [19]:
merged_ecg = df_notes_ts.loc[df_notes_ts.CATEGORY == 'ECG'].groupby(['pat_id','CHARTDATE']).TEXT.agg(lambda s: ' [SEP] '.join(s))
merged_echo = df_notes_ts.loc[df_notes_ts.CATEGORY == 'Echo'].groupby(['pat_id','CHARTDATE']).TEXT.agg(lambda s: ' [SEP] '.join(s))

In [20]:
df_ecg = df_notes_ts.loc[df_notes_ts.CATEGORY == 'ECG']
df_ecg = df_ecg.reset_index().groupby(['pat_id','CHARTDATE']).first()
df_ecg.TEXT = merged_ecg.values
df_ecg = df_ecg.reset_index().set_index('pat_id')

In [21]:
df_echo = df_notes_ts.loc[df_notes_ts.CATEGORY == 'Echo']
df_echo = df_echo.reset_index().groupby(['pat_id','CHARTDATE']).first()
df_echo.TEXT = merged_echo.values
df_echo = df_echo.reset_index().set_index('pat_id')

In [22]:
df_dateonly = pd.concat([df_echo, df_ecg])
df_dateonly['hours_in'] = ((df_dateonly.CHARTDATE - df_dateonly.INDATE) / pd.Timedelta(hours=1)).astype(int)

### Create Hours In attribute

In [23]:
df_notes_ts = df_notes_ts.loc[~df_notes_ts.CATEGORY.map(lambda c: c in ['ECG','Echo']), :]
df_notes_ts['hours_in'] = ((df_notes_ts.CHARTTIME - df_notes_ts.INTIME) / pd.Timedelta(hours=1)).astype(int)

In [24]:
df_notes_ts = pd.concat([df_notes_ts, df_dateonly])

### Trim time series

In [25]:
df_notes_ts = df_notes_ts.reset_index().set_index(['pat_id','hours_in'])[['CATEGORY','TEXT']]

In [26]:
df_notes_ts.sort_values(by=['pat_id','hours_in'], inplace=True)

### One Hot Encode categories

In [27]:
df_notes_ts.loc[df_notes_ts.CATEGORY == 'Nursing/other', 'CATEGORY'] = 'Nursing'
df_cats = pd.get_dummies(df_notes_ts.CATEGORY, prefix='cat')
df_notes_ts = df_notes_ts.join(df_cats).drop('CATEGORY', axis=1)

### Preprocess text

In [28]:
def prepare_text(t):
    t_pre = re.sub('(\n+|\.|\?\!)', ' [SEP] ', t)
    t_pre = re.sub('\s+', ' ', t_pre)
    t_pre = re.sub('\s(\[SEP\]\s){2,}', ' [SEP] ', t_pre)
    t_pre = t_pre.strip()
    t_pre = re.sub('[^a-zA-Z0-9_\[\] ]', '', t_pre)
    
    return t_pre

In [29]:
df_notes_static.TEXT = df_notes_static.TEXT.map(lambda t: t if pd.isnull(t) else prepare_text(t))

In [30]:
df_notes_ts.TEXT = df_notes_ts.TEXT.map(lambda t: t if pd.isnull(t) else prepare_text(t))

### Split & Save Results

In [31]:
processed_dir = '../data/processed/'
csv_static_filename = 'notes_static.csv'
csv_ts_filename = 'notes_ts.csv'

In [32]:
train_idxs = set(np.load(os.path.join(processed_dir, 'train_idxs.npy')))
test_idxs = set(np.load(os.path.join(processed_dir, 'test_idxs.npy')))

In [33]:
static_idxs = set(df_notes_static.index.values)
ts_idxs = set(df_notes_ts.index.get_level_values(0).values)

In [34]:
train_idxs_static = list(static_idxs & train_idxs)
test_idxs_static = list(static_idxs & test_idxs)

In [35]:
train_idxs_ts = list(ts_idxs & train_idxs)
test_idxs_ts = list(ts_idxs & test_idxs)

In [36]:
df_static_train = df_notes_static.loc[train_idxs_static].copy()
df_static_test = df_notes_static.loc[test_idxs_static].copy()

In [37]:
df_ts_train = df_notes_ts.loc[train_idxs_ts].copy()
df_ts_test = df_notes_ts.loc[test_idxs_ts].copy()

In [38]:
df_static_train.to_csv(os.path.join(processed_dir, 'train/', csv_static_filename))
df_static_test.to_csv(os.path.join(processed_dir, 'test/', csv_static_filename))

In [39]:
df_ts_train.to_csv(os.path.join(processed_dir, 'train/', csv_ts_filename))
df_ts_test.to_csv(os.path.join(processed_dir, 'test/', csv_ts_filename))